In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

In [2]:
# Load the dataset
df = pd.read_csv('../data/enriched/HB_PSEUDO_clean_elevation_soil.csv')

# Remove location and time-specific columns (following your approach)
df = df.drop(columns=['gbifID', 'gridmet_lat', 'gridmet_lon', 'gridmet_date', 'decimalLatitude', 'decimalLongitude'])

# Convert datetime to numeric by extracting components with error handling
def safe_datetime_parse(datetime_series):
    """Safely parse datetime series with mixed formats"""
    parsed_dates = []
    
    for date_str in datetime_series:
        try:
            # Try parsing as full datetime first
            parsed = pd.to_datetime(date_str)
            parsed_dates.append(parsed)
        except:
            try:
                # If that fails, try parsing as date only
                parsed = pd.to_datetime(date_str, format='%Y-%m-%d')
                parsed_dates.append(parsed)
            except:
                # If all else fails, use NaT (Not a Time)
                parsed_dates.append(pd.NaT)
    
    return pd.Series(parsed_dates, index=datetime_series.index)

# Parse the datetime column safely
df['parsed_datetime'] = safe_datetime_parse(df['datetime'])

# Extract only season (not specific month/day to avoid overfitting)
df['month'] = df['parsed_datetime'].dt.month 

def month_to_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

# Apply the function to create a new 'season' column
df['season'] = df['month'].apply(month_to_season)

# Encode season as numbers
season_map = {'Winter': 0, 'Spring': 1, 'Summer': 2, 'Fall': 3}
df['season_num'] = df['season'].map(season_map)

# Drop datetime columns and keep only season_num
df = df.drop(['datetime', 'parsed_datetime', 'season', 'month'], axis=1)

# Check for any remaining NaN values and drop them
print(f"Shape before dropping NaN: {df.shape}")
df.dropna(inplace=True)
print(f"Shape after dropping NaN: {df.shape}")
print(f"Positive samples: {df['occurrence'].sum()}")
print(f"Negative samples: {(df['occurrence'] == 0).sum()}")

Shape before dropping NaN: (6477, 12)
Shape after dropping NaN: (6240, 12)
Positive samples: 1646
Negative samples: 4594


In [3]:
# Add only a few key interaction features that make ecological sense
df['elevation_temp_interaction'] = df['elevation'] * df['air_temperature']
df['temp_precip_interaction'] = df['air_temperature'] * df['precipitation_amount']

# Simple suitability score based on elevation (huckleberries prefer mid-elevations)
df['elevation_suitability'] = np.where(
    (df['elevation'] >= 1000) & (df['elevation'] <= 2500), 1.0,
    np.where((df['elevation'] >= 500) & (df['elevation'] <= 3000), 0.7, 0.3)
)

print("Feature engineering completed")

Feature engineering completed


In [4]:
# Separate features and target
X = df.drop('occurrence', axis=1)
y = df['occurrence']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Create and train the model (following your approach)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Calibrate the model to get better probability estimates
rf_calibrated = CalibratedClassifierCV(rf, cv=5, method='isotonic')
rf_calibrated.fit(X_train, y_train)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

Training set size: (4992, 14)
Test set size: (1248, 14)


In [5]:
# Make predictions
y_pred = rf_calibrated.predict(X_test)
y_proba = rf_calibrated.predict_proba(X_test)[:, 1]

# Print results
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred))

print(f"\nROC AUC: {roc_auc_score(y_test, y_proba):.3f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       919
           1       0.87      0.91      0.89       329

    accuracy                           0.94      1248
   macro avg       0.92      0.93      0.92      1248
weighted avg       0.94      0.94      0.94      1248


ROC AUC: 0.972

Top 10 Most Important Features:
                                      feature  importance
9                                     soil_ph    0.284459
6   surface_downwelling_shortwave_flux_in_air    0.103822
11                 elevation_temp_interaction    0.099110
2                           specific_humidity    0.096269
8                                   elevation    0.095275
5                potential_evapotranspiration    0.065585
10                                 season_num    0.061067
13                      elevation_suitability    0.047876
4                 mean_vapor_pressure_deficit    0.040919
0      

In [6]:
# Create synthetic test coordinates
np.random.seed(42)
test_coords = []
for _ in range(1000):
    elevation = np.random.uniform(500, 3000)
    air_temperature = np.random.uniform(280, 310)  # Kelvin range
    precipitation_amount = np.random.uniform(30, 300)
    soil_ph = np.random.uniform(5.5, 7.5)
    specific_humidity = np.random.uniform(0.001, 0.015)
    relative_humidity = np.random.uniform(20, 100)
    mean_vapor_pressure_deficit = np.random.uniform(0.1, 2.5)
    potential_evapotranspiration = np.random.uniform(0.5, 10)
    surface_downwelling_shortwave_flux_in_air = np.random.uniform(50, 350)
    wind_speed = np.random.uniform(1, 10)
    season_num = np.random.randint(0, 4)
    
    test_coords.append({
        'elevation': elevation,
        'air_temperature': air_temperature,
        'precipitation_amount': precipitation_amount,
        'soil_ph': soil_ph,
        'specific_humidity': specific_humidity,
        'relative_humidity': relative_humidity,
        'mean_vapor_pressure_deficit': mean_vapor_pressure_deficit,
        'potential_evapotranspiration': potential_evapotranspiration,
        'surface_downwelling_shortwave_flux_in_air': surface_downwelling_shortwave_flux_in_air,
        'wind_speed': wind_speed,
        'season_num': season_num
    })

test_df = pd.DataFrame(test_coords)

# Add the engineered features
test_df['elevation_temp_interaction'] = test_df['elevation'] * test_df['air_temperature']
test_df['temp_precip_interaction'] = test_df['air_temperature'] * test_df['precipitation_amount']
test_df['elevation_suitability'] = np.where(
    (test_df['elevation'] >= 1000) & (test_df['elevation'] <= 2500), 1.0,
    np.where((test_df['elevation'] >= 500) & (test_df['elevation'] <= 3000), 0.7, 0.3)
)

# Fix the feature mismatch issue
# Get the exact column order from the training data
expected_columns = X.columns.tolist()
print(f"Expected columns: {expected_columns}")
print(f"Test data columns: {list(test_df.columns)}")

# Check for missing columns
missing_columns = set(expected_columns) - set(test_df.columns)
if missing_columns:
    print(f"Missing columns: {missing_columns}")
    # Add missing columns with default values
    for col in missing_columns:
        test_df[col] = 0.0

# Reorder test data columns to match training data exactly
test_df = test_df[expected_columns]

print(f"Final test data shape: {test_df.shape}")
print(f"Final test data columns: {list(test_df.columns)}")

# Make predictions
synthetic_proba = rf_calibrated.predict_proba(test_df)[:, 1]

print(f"Synthetic test results:")
print(f"Mean probability: {synthetic_proba.mean():.3f}")
print(f"Max probability: {synthetic_proba.max():.3f}")
print(f"Min probability: {synthetic_proba.min():.3f}")
print(f"High probability samples (>0.6): {(synthetic_proba > 0.6).sum()}")
print(f"Medium probability samples (0.3-0.6): {((synthetic_proba >= 0.3) & (synthetic_proba <= 0.6)).sum()}")
print(f"Low probability samples (<0.3): {(synthetic_proba < 0.3).sum()}")

Expected columns: ['air_temperature', 'precipitation_amount', 'specific_humidity', 'relative_humidity', 'mean_vapor_pressure_deficit', 'potential_evapotranspiration', 'surface_downwelling_shortwave_flux_in_air', 'wind_speed', 'elevation', 'soil_ph', 'season_num', 'elevation_temp_interaction', 'temp_precip_interaction', 'elevation_suitability']
Test data columns: ['elevation', 'air_temperature', 'precipitation_amount', 'soil_ph', 'specific_humidity', 'relative_humidity', 'mean_vapor_pressure_deficit', 'potential_evapotranspiration', 'surface_downwelling_shortwave_flux_in_air', 'wind_speed', 'season_num', 'elevation_temp_interaction', 'temp_precip_interaction', 'elevation_suitability']
Final test data shape: (1000, 14)
Final test data columns: ['air_temperature', 'precipitation_amount', 'specific_humidity', 'relative_humidity', 'mean_vapor_pressure_deficit', 'potential_evapotranspiration', 'surface_downwelling_shortwave_flux_in_air', 'wind_speed', 'elevation', 'soil_ph', 'season_num', 'e

In [8]:
import joblib

# Save the calibrated model and feature columns
model_data = {
    'rf_calibrated': rf_calibrated,
    'feature_columns': X.columns.tolist(),
    'feature_importance': feature_importance
}

joblib.dump(model_data, '../models/improved_huckleberry_model.pkl')

print("Improved model saved successfully!")

Improved model saved successfully!


In [13]:
import numpy as np
import pandas as pd
import xarray as xr
import pystac_client
import planetary_computer
import requests
import time
from tqdm import tqdm

# Your coordinates (lat, lon pairs)
coordinates = [
    (44.5, -116.5), (44.6, -116.4), (44.7, -116.3), (44.499669, -111.329396),
    (48.687832, -116.924988), (48.698033, -116.953418), (48.721255, -116.973486),
    (48.769863, -116.948780), (48.802889, -116.983530), (48.818715, -116.936533),
    (48.790450, -117.027455), (48.815164, -116.981784), (48.789158, -116.957771),
    (48.796396, -116.970464), (48.785087, -116.933929), (48.805554, -116.888647),
    (48.751289, -116.994538)
]

# Create DataFrame with coordinates
coords_df = pd.DataFrame(coordinates, columns=['decimalLatitude', 'decimalLongitude'])

print(f"Testing {len(coords_df)} coordinates...")

# Connect to Planetary Computer and load GridMET (following your pattern)
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
asset = catalog.get_collection("gridmet").assets["zarr-abfs"]
ds = xr.open_zarr(
    asset.href,
    storage_options=asset.extra_fields["xarray:storage_options"],
    **asset.extra_fields["xarray:open_kwargs"],
    chunks="auto"
)

# Get the most recent date available
latest_time = ds.time.values.max()
print(f"Using latest available date: {pd.to_datetime(latest_time).date()}")

# Variables to extract (matching your pattern)
VARS_TO_EXTRACT = [
    "air_temperature",
    "precipitation_amount", 
    "specific_humidity",
    "relative_humidity",
    "mean_vapor_pressure_deficit",
    "potential_evapotranspiration",
    "surface_downwelling_shortwave_flux_in_air",
    "wind_speed"
]

# Snap coordinates to nearest GridMET grid (following your pattern)
def snap_to_grid(val, grid):
    return grid[np.abs(grid - val).argmin()]

lats = coords_df['decimalLatitude'].values
lons = coords_df['decimalLongitude'].values

snapped_lats = np.array([snap_to_grid(lat, ds.lat.values) for lat in lats])
snapped_lons = np.array([snap_to_grid(lon, ds.lon.values) for lon in lons])

# Build DataFrame
test_df = pd.DataFrame({
    "decimalLatitude": snapped_lats,
    "decimalLongitude": snapped_lons
})

# Extract all variables at once for the latest date
for var in VARS_TO_EXTRACT:
    print(f"Extracting {var}...")
    values = ds[var].sel(
        time=latest_time,
        lat=xr.DataArray(snapped_lats, dims="points"),
        lon=xr.DataArray(snapped_lons, dims="points")
    ).values
    test_df[var] = values

# Add the gridmet_date column
test_df['gridmet_date'] = pd.to_datetime(latest_time)

print("✅ GridMET data extracted successfully!")

# Now get elevation data (following your elevation.py pattern)
def get_elevation(lat, lon):
    """Query Open-Elevation API for elevation."""
    try:
        response = requests.get("https://api.open-elevation.com/api/v1/lookup", 
                              params={"locations": f"{lat},{lon}"})
        if response.status_code == 200:
            elevation = response.json()["results"][0]["elevation"]
            return elevation
        else:
            print(f"⚠️ API error: {response.status_code}")
            return 1000  # Default elevation
    except Exception as e:
        print(f"❌ Failed to get elevation for {lat}, {lon}: {e}")
        return 1000  # Default elevation

print("Getting elevation data...")
elevations = []
for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Getting elevations"):
    lat, lon = row['decimalLatitude'], row['decimalLongitude']
    elevation = get_elevation(lat, lon)
    elevations.append(elevation)
    time.sleep(0.5)  # Be polite to API

test_df['elevation'] = elevations

# Get soil pH data (following your soil_data.py pattern)
def get_soil_ph(lat, lon, retries=3, base_delay=2):
    """Fetch soil pH with retries."""
    url = f"https://rest.isric.org/soilgrids/v2.0/properties/query?lat={lat}&lon={lon}&property=phh2o&depth=15-30cm&value=mean"
    
    for attempt in range(retries):
        try:
            response = requests.get(url)
            if response.status_code == 429:
                wait = base_delay * (2 ** attempt)
                print(f"⏳ Rate limited. Waiting {wait}s...")
                time.sleep(wait)
                continue
            
            response.raise_for_status()
            data = response.json()
            layers = data.get("properties", {}).get("layers", [])
            for layer in layers:
                if layer["name"] == "phh2o":
                    values = layer["depths"][0]["values"]
                    raw_ph = values.get("mean")
                    if raw_ph is not None:
                        return round(raw_ph / 10.0, 2)  # Convert to pH scale
            return 6.5  # Default pH
            
        except Exception as e:
            print(f"⚠️ Error at ({lat}, {lon}): {e}")
            time.sleep(base_delay * (2 ** attempt))
    
    return 6.5  # Default pH

print("Getting soil pH data...")
soil_ph_values = []
for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Getting soil pH"):
    lat, lon = row['decimalLatitude'], row['decimalLongitude']
    ph = get_soil_ph(lat, lon)
    soil_ph_values.append(ph)
    time.sleep(0.5)  # Be polite to API

test_df['soil_ph'] = soil_ph_values

print("✅ Environmental data extraction complete!")

# Now apply the same transformations as your training data
# Parse datetime and create season features
test_df['parsed_datetime'] = pd.to_datetime(test_df['gridmet_date'])
test_df['month'] = test_df['parsed_datetime'].dt.month

def month_to_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

test_df['season'] = test_df['month'].apply(month_to_season)
season_map = {'Winter': 0, 'Spring': 1, 'Summer': 2, 'Fall': 3}
test_df['season_num'] = test_df['season'].map(season_map)

# Add engineered features
test_df['elevation_temp_interaction'] = test_df['elevation'] * test_df['air_temperature']
test_df['temp_precip_interaction'] = test_df['air_temperature'] * test_df['precipitation_amount']
test_df['elevation_suitability'] = np.where(
    (test_df['elevation'] >= 1000) & (test_df['elevation'] <= 2500), 1.0,
    np.where((test_df['elevation'] >= 500) & (test_df['elevation'] <= 3000), 0.7, 0.3)
)

# Drop unnecessary columns and reorder to match training data
test_df = test_df.drop(['parsed_datetime', 'season', 'month', 'gridmet_date'], axis=1)

# Ensure we have the same columns as training data
expected_columns = X.columns.tolist()
missing_columns = set(expected_columns) - set(test_df.columns)
if missing_columns:
    print(f"Adding missing columns: {missing_columns}")
    for col in missing_columns:
        test_df[col] = 0.0

# Reorder columns to match training data
test_df = test_df[expected_columns]

print(f"Final test data shape: {test_df.shape}")
print(f"Features: {list(test_df.columns)}")

# Make predictions
predictions = rf_calibrated.predict(test_df)
probabilities = rf_calibrated.predict_proba(test_df)[:, 1]

# Add predictions back to original coordinates
results_df = coords_df.copy()
results_df['prediction'] = predictions
results_df['probability'] = probabilities

print("\nPrediction Results:")
print("=" * 60)
for idx, row in results_df.iterrows():
    lat, lon = row['decimalLatitude'], row['decimalLongitude']
    pred = "HUCKLEBERRY" if row['prediction'] == 1 else "NO HUCKLEBERRY"
    prob = row['probability']
    print(f"({lat:.6f}, {lon:.6f}): {pred} (Probability: {prob:.3f})")

print(f"\nSummary:")
print(f"High probability (>0.6): {(probabilities > 0.6).sum()}")
print(f"Medium probability (0.3-0.6): {((probabilities >= 0.3) & (probabilities <= 0.6)).sum()}")
print(f"Low probability (<0.3): {(probabilities < 0.3).sum()}")
print(f"Mean probability: {probabilities.mean():.3f}")

Testing 17 coordinates...
Using latest available date: 2020-12-31
Extracting air_temperature...
Extracting precipitation_amount...
Extracting specific_humidity...
Extracting relative_humidity...
Extracting mean_vapor_pressure_deficit...
Extracting potential_evapotranspiration...
Extracting surface_downwelling_shortwave_flux_in_air...
Extracting wind_speed...
✅ GridMET data extracted successfully!
Getting elevation data...


Getting elevations: 100%|██████████| 17/17 [00:11<00:00,  1.46it/s]


Getting soil pH data...


Getting soil pH:  88%|████████▊ | 15/17 [00:12<00:01,  1.29it/s]

⏳ Rate limited. Waiting 2s...


Getting soil pH: 100%|██████████| 17/17 [00:15<00:00,  1.07it/s]

✅ Environmental data extraction complete!
Final test data shape: (17, 14)
Features: ['air_temperature', 'precipitation_amount', 'specific_humidity', 'relative_humidity', 'mean_vapor_pressure_deficit', 'potential_evapotranspiration', 'surface_downwelling_shortwave_flux_in_air', 'wind_speed', 'elevation', 'soil_ph', 'season_num', 'elevation_temp_interaction', 'temp_precip_interaction', 'elevation_suitability']

Prediction Results:
(44.500000, -116.500000): NO HUCKLEBERRY (Probability: 0.011)
(44.600000, -116.400000): NO HUCKLEBERRY (Probability: 0.011)
(44.700000, -116.300000): NO HUCKLEBERRY (Probability: 0.014)
(44.499669, -111.329396): NO HUCKLEBERRY (Probability: 0.006)
(48.687832, -116.924988): NO HUCKLEBERRY (Probability: 0.029)
(48.698033, -116.953418): NO HUCKLEBERRY (Probability: 0.029)
(48.721255, -116.973486): NO HUCKLEBERRY (Probability: 0.031)
(48.769863, -116.948780): NO HUCKLEBERRY (Probability: 0.055)
(48.802889, -116.983530): NO HUCKLEBERRY (Probability: 0.022)
(48.81871

In [16]:
# Test with summer 2019 conditions
print("=== TESTING WITH SUMMER 2019 CONDITIONS ===")

# Use summer 2019 date
summer_time = pd.Timestamp('2019-07-15')  # Mid-summer 2019
print(f"Using summer date: {summer_time.date()}")

# Re-extract GridMET data for summer 2019
summer_test_df = pd.DataFrame({
    "decimalLatitude": snapped_lats,
    "decimalLongitude": snapped_lons
})

# Extract summer data
for var in VARS_TO_EXTRACT:
    print(f"Extracting {var} for summer 2019...")
    values = ds[var].sel(
        time=summer_time,
        lat=xr.DataArray(snapped_lats, dims="points"),
        lon=xr.DataArray(snapped_lons, dims="points")
    ).values
    summer_test_df[var] = values

# Add summer date
summer_test_df['gridmet_date'] = summer_time

# Reuse elevation and soil data from before
summer_test_df['elevation'] = test_df['elevation'].values
summer_test_df['soil_ph'] = test_df['soil_ph'].values

# Apply transformations for summer
summer_test_df['parsed_datetime'] = pd.to_datetime(summer_test_df['gridmet_date'])
summer_test_df['month'] = summer_test_df['parsed_datetime'].dt.month
summer_test_df['season'] = summer_test_df['month'].apply(month_to_season)
summer_test_df['season_num'] = summer_test_df['season'].map(season_map)

# Add engineered features
summer_test_df['elevation_temp_interaction'] = summer_test_df['elevation'] * summer_test_df['air_temperature']
summer_test_df['temp_precip_interaction'] = summer_test_df['air_temperature'] * summer_test_df['precipitation_amount']
summer_test_df['elevation_suitability'] = np.where(
    (summer_test_df['elevation'] >= 1000) & (summer_test_df['elevation'] <= 2500), 1.0,
    np.where((summer_test_df['elevation'] >= 500) & (summer_test_df['elevation'] <= 3000), 0.7, 0.3)
)

# Drop unnecessary columns
summer_test_df = summer_test_df.drop(['parsed_datetime', 'season', 'month', 'gridmet_date'], axis=1)

# Ensure same columns as training data
missing_columns = set(expected_columns) - set(summer_test_df.columns)
if missing_columns:
    for col in missing_columns:
        summer_test_df[col] = 0.0

summer_test_df = summer_test_df[expected_columns]

# Make predictions
summer_predictions = rf_calibrated.predict(summer_test_df)
summer_probabilities = rf_calibrated.predict_proba(summer_test_df)[:, 1]

# Show results
print("\n=== SUMMER 2019 PREDICTION RESULTS ===")
print("=" * 60)
for idx, row in coords_df.iterrows():
    lat, lon = row['decimalLatitude'], row['decimalLongitude']
    pred = "HUCKLEBERRY" if summer_predictions[idx] == 1 else "NO HUCKLEBERRY"
    prob = summer_probabilities[idx]
    print(f"({lat:.6f}, {lon:.6f}): {pred} (Probability: {prob:.3f})")

print(f"\nSummer 2019 Summary:")
print(f"High probability (>0.6): {(summer_probabilities > 0.6).sum()}")
print(f"Medium probability (0.3-0.6): {((summer_probabilities >= 0.3) & (summer_probabilities <= 0.6)).sum()}")
print(f"Low probability (<0.3): {(summer_probabilities < 0.3).sum()}")
print(f"Mean probability: {summer_probabilities.mean():.3f}")

# Compare summer vs winter data
print(f"\n=== COMPARISON ===")
print(f"Winter mean probability: {probabilities.mean():.3f}")
print(f"Summer 2019 mean probability: {summer_probabilities.mean():.3f}")
print(f"Improvement: {((summer_probabilities.mean() - probabilities.mean()) / probabilities.mean() * 100):.1f}%")

# Show some sample summer data
print(f"\n=== SUMMER 2019 DATA SAMPLE ===")
print(f"Temperature range: {summer_test_df['air_temperature'].min():.1f}K to {summer_test_df['air_temperature'].max():.1f}K")
print(f"Solar radiation range: {summer_test_df['surface_downwelling_shortwave_flux_in_air'].min():.1f} to {summer_test_df['surface_downwelling_shortwave_flux_in_air'].max():.1f} W/m²")
print(f"Season: {summer_test_df['season_num'].iloc[0]} (Summer)")
print(f"Precipitation range: {summer_test_df['precipitation_amount'].min():.1f} to {summer_test_df['precipitation_amount'].max():.1f} mm")

=== TESTING WITH SUMMER 2019 CONDITIONS ===
Using summer date: 2019-07-15
Extracting air_temperature for summer 2019...
Extracting precipitation_amount for summer 2019...
Extracting specific_humidity for summer 2019...
Extracting relative_humidity for summer 2019...
Extracting mean_vapor_pressure_deficit for summer 2019...
Extracting potential_evapotranspiration for summer 2019...
Extracting surface_downwelling_shortwave_flux_in_air for summer 2019...
Extracting wind_speed for summer 2019...

=== SUMMER 2019 PREDICTION RESULTS ===
(44.500000, -116.500000): NO HUCKLEBERRY (Probability: 0.258)
(44.600000, -116.400000): NO HUCKLEBERRY (Probability: 0.378)
(44.700000, -116.300000): HUCKLEBERRY (Probability: 0.963)
(44.499669, -111.329396): HUCKLEBERRY (Probability: 0.920)
(48.687832, -116.924988): HUCKLEBERRY (Probability: 0.961)
(48.698033, -116.953418): HUCKLEBERRY (Probability: 0.961)
(48.721255, -116.973486): HUCKLEBERRY (Probability: 0.916)
(48.769863, -116.948780): HUCKLEBERRY (Proba

In [21]:
# Debug: Check what features the original model expects
print("=== DEBUGGING ORIGINAL MODEL FEATURES ===")

# Check what features the original model was trained on
if hasattr(original_model, 'feature_names_in_'):
    original_features = list(original_model.feature_names_in_)
    print(f"Original model expects these features (in order):")
    for i, feature in enumerate(original_features):
        print(f"  {i}: {feature}")
    print(f"Total features: {len(original_features)}")
else:
    print("❌ Original model doesn't have feature_names_in_ attribute")
    # Try to get features from the saved model data
    if isinstance(original_model_data, dict) and 'feature_names' in original_model_data:
        original_features = original_model_data['feature_names']
        print(f"Original model features from saved data:")
        for i, feature in enumerate(original_features):
            print(f"  {i}: {feature}")
    else:
        print("❌ Could not determine original model features")
        original_features = None

# Check what features we're providing
print(f"\nFeatures we're providing:")
for i, feature in enumerate(original_compatible_df.columns):
    print(f"  {i}: {feature}")
print(f"Total features: {len(original_compatible_df.columns)}")

# Compare the two lists
if original_features is not None:
    print(f"\n=== FEATURE COMPARISON ===")
    print(f"Missing from our data: {set(original_features) - set(original_compatible_df.columns)}")
    print(f"Extra in our data: {set(original_compatible_df.columns) - set(original_features)}")
    
    # Create the correct feature set
    print(f"\n=== CREATING CORRECT FEATURE SET ===")
    correct_df = pd.DataFrame()
    
    for feature in original_features:
        if feature in original_compatible_df.columns:
            correct_df[feature] = original_compatible_df[feature]
        else:
            # Add missing features with default values
            print(f"Adding missing feature '{feature}' with default value")
            if feature in ['year', 'month', 'day']:
                correct_df[feature] = 2019 if feature == 'year' else 7 if feature == 'month' else 15
            elif feature == 'season_num':
                correct_df[feature] = 2  # Summer
            else:
                correct_df[feature] = 0.0  # Default for other features
    
    print(f"Correct feature set shape: {correct_df.shape}")
    print(f"Correct features: {list(correct_df.columns)}")
    
    # Now test the original model
    print(f"\n=== TESTING ORIGINAL MODEL WITH CORRECT FEATURES ===")
    try:
        original_predictions = original_model.predict(correct_df)
        original_probabilities = original_model.predict_proba(correct_df)[:, 1]
        print("✅ Original model predictions successful!")
        
        # Now we can do the comparison
        print(f"\n=== MODEL COMPARISON RESULTS ===")
        print("=" * 80)
        
        # Test improved model (with engineered features)
        improved_predictions = rf_calibrated.predict(summer_test_df)
        improved_probabilities = rf_calibrated.predict_proba(summer_test_df)[:, 1]
        
        # Display results side by side
        print(f"{'Location':<25} {'Improved Model':<20} {'Original Model':<20} {'Difference':<15}")
        print(f"{'':<25} {'Pred':<8} {'Prob':<8} {'Pred':<8} {'Prob':<8} {'Prob Diff':<15}")
        print("-" * 80)
        
        for idx, row in coords_df.iterrows():
            lat, lon = row['decimalLatitude'], row['decimalLongitude']
            location = f"({lat:.3f}, {lon:.3f})"
            
            # Improved model results
            improved_pred = "HUCKLE" if improved_predictions[idx] == 1 else "NO"
            improved_prob = improved_probabilities[idx]
            
            # Original model results
            original_pred = "HUCKLE" if original_predictions[idx] == 1 else "NO"
            original_prob = original_probabilities[idx]
            
            # Difference
            prob_diff = improved_prob - original_prob
            
            print(f"{location:<25} {improved_pred:<8} {improved_prob:<8.3f} {original_pred:<8} {original_prob:<8.3f} {prob_diff:<+8.3f}")
        
        # Summary statistics
        print(f"\n=== SUMMARY ===")
        print(f"Improved model mean probability: {improved_probabilities.mean():.3f}")
        print(f"Original model mean probability: {original_probabilities.mean():.3f}")
        print(f"Improvement: {((improved_probabilities.mean() - original_probabilities.mean()) / original_probabilities.mean() * 100):.1f}%")
        
    except Exception as e:
        print(f"❌ Still getting error: {e}")
        print("Please check the original model file or provide more details about how it was trained")

else:
    print("❌ Cannot proceed without knowing the original model's expected features")

=== DEBUGGING ORIGINAL MODEL FEATURES ===
Original model expects these features (in order):
  0: air_temperature
  1: precipitation_amount
  2: specific_humidity
  3: relative_humidity
  4: mean_vapor_pressure_deficit
  5: potential_evapotranspiration
  6: surface_downwelling_shortwave_flux_in_air
  7: wind_speed
  8: elevation
  9: soil_ph
  10: year
  11: month
  12: day
  13: season_num
Total features: 14

Features we're providing:
  0: year
  1: month
  2: day
  3: air_temperature
  4: precipitation_amount
  5: specific_humidity
  6: relative_humidity
  7: mean_vapor_pressure_deficit
  8: potential_evapotranspiration
  9: surface_downwelling_shortwave_flux_in_air
  10: wind_speed
  11: elevation
  12: soil_ph
  13: season_num
Total features: 14

=== FEATURE COMPARISON ===
Missing from our data: set()
Extra in our data: set()

=== CREATING CORRECT FEATURE SET ===
Correct feature set shape: (17, 14)
Correct features: ['air_temperature', 'precipitation_amount', 'specific_humidity', 'r